In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('clients_churn').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/07/06 08:24:30 WARN Utils: Your hostname, predator resolves to a loopback address: 127.0.1.1; using 172.20.67.13 instead (on interface eth0)
23/07/06 08:24:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 08:24:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
data = spark.read.csv('client_data.csv', sep=',', header=True, inferSchema=True)

In [5]:
data.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [6]:
data.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [7]:
binColumns = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [8]:
from pyspark.sql import functions as f

In [9]:
allColumns = [f.when(f.col(c)=='Sim', 1).otherwise(0).alias(c) for c in binColumns]

In [10]:
for col in reversed(data.columns):
    if col not in binColumns:
        allColumns.insert(0, col)

allColumns 

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [11]:
data.select(allColumns).show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [12]:
dataset = data.select(allColumns)

In [13]:
data.select(['Internet','TipoContrato','MetodoPagamento']).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [14]:
dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0).show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [15]:
Internet = dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

In [16]:
dataset = dataset.join(
    Internet, 'id', how='inner'
).join(
    TipoContrato, 'id', how='inner'
).join(
    MetodoPagamento, 'id', how='inner'
).select(
    '*',
    f.col('DSL').alias('Internet_DSL'),
    f.col('FibraOptica').alias('Internet_FibraOptica'),
    f.col('Nao').alias('Internet_Nao'),
    f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
    f.col('UmAno').alias('TipoContrato_UmAno'),
    f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
    f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
    f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
    f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
    f.col('Boleto').alias('MetodoPagamento_Boleto'),
).drop(
    'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 
    'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 
    'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
)

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
dataset = dataset.withColumnRenamed('Churn', 'label')

In [19]:
X = dataset.columns
X.remove('label')
X.remove('id')
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [20]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [21]:
dataset_prep = assembler.transform(dataset= dataset).select('features', 'label')

In [22]:
dataset_prep.show()

23/07/06 08:24:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----+
|            features|label|
+--------------------+-----+
|(24,[1,2,11,12,13...|    1|
|(24,[1,2,3,5,6,8,...|    1|
|(24,[1,2,5,6,10,1...|    0|
|(24,[1,2,3,5,8,12...|    0|
|(24,[1,2,3,5,6,11...|    1|
|(24,[1,2,5,6,12,1...|    1|
|(24,[1,2,3,5,6,8,...|    0|
|(24,[1,2,5,6,15,1...|    0|
|(24,[1,2,3,5,7,8,...|    0|
|(24,[1,2,3,5,12,1...|    1|
|(24,[1,2,5,16,17,...|    0|
|(24,[1,2,5,8,12,1...|    0|
|(24,[1,2,3,5,6,11...|    1|
|(24,[1,2,5,6,13,1...|    0|
|(24,[1,2,5,6,8,11...|    1|
|(24,[0,1,2,3,5,6,...|    1|
|(24,[0,1,2,3,5,7,...|    0|
|(24,[1,2,5,8,14,1...|    1|
|(24,[1,2,5,16,19,...|    0|
|(24,[1,2,3,4,5,11...|    1|
+--------------------+-----+
only showing top 20 rows



In [23]:
train, test = dataset_prep.randomSplit([0.7, 0.3], seed=42)

In [24]:
from pyspark.ml.classification import LogisticRegression

In [25]:
lr = LogisticRegression()

In [26]:
lr_model = lr.fit(train)

23/07/06 08:31:15 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/06 08:31:15 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [27]:
predictions_lr_train = lr_model.transform(train)

In [29]:
summary_lr_model = lr_model.summary

In [36]:
print(f'accuracy: {summary_lr_model.accuracy * 100}%')
print(f'precision: {summary_lr_model.precisionByLabel[1] * 100}%')
print(f'recall: {summary_lr_model.recallByLabel[1] * 100}%')
print(f'F1: {summary_lr_model.fMeasureByLabel()[1] * 100}%')

accuracy: 78.33424060968972%
precision: 76.93490517683239%
recall: 81.26691932864104%
F1: 79.0416008425487%


In [43]:
tp = predictions_lr_train.select('label', 'prediction').where((f.col('label')== 1) & (f.col('prediction') == 1)).count()
fp = predictions_lr_train.select('label', 'prediction').where((f.col('label')== 0) & (f.col('prediction') == 1)).count()
fn = predictions_lr_train.select('label', 'prediction').where((f.col('label')== 1) & (f.col('prediction') == 0)).count()
tn = predictions_lr_train.select('label', 'prediction').where((f.col('label')== 0) & (f.col('prediction') == 0)).count()
print(tp, fp, fn, tn)

3002 900 692 2754


In [44]:
from pyspark.ml.classification import DecisionTreeClassifier

In [46]:
dtc = DecisionTreeClassifier(seed=42)
dtc_model = dtc.fit(train)

In [47]:
dtc_predictions_train = dtc_model.transform(train)
dtc_prediction_test = dtc_model.transform(test)

In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()
evaluator.evaluate(dtc_prediction_test, {evaluator.metricName: 'accuracy'})

0.7866666666666666

In [50]:
print("Acurácia: %f" % evaluator.evaluate(dtc_prediction_test, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(dtc_prediction_test, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(dtc_prediction_test, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(dtc_prediction_test, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.786667
Precisão: 0.797030
Recall: 0.761486
F1: 0.778853


In [51]:
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(seed=42, numTrees=10)

In [52]:
rfc_model = rfc.fit(train)

In [53]:
rfc_predictions_train = rfc_model.transform(train)
rfc_predictions_test = rfc_model.transform(test)

In [54]:
print("Acurácia: %f" % evaluator.evaluate(rfc_predictions_test, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(rfc_predictions_test, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(rfc_predictions_test, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(rfc_predictions_test, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.780667
Precisão: 0.771108
Recall: 0.789865
F1: 0.780374


In [55]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [56]:
dtc = DecisionTreeClassifier(seed=42)
grid = ParamGridBuilder().addGrid(dtc.maxDepth, [2,5, 10]).addGrid(dtc.maxBins, [10,20,30]).build()
evaluator = MulticlassClassificationEvaluator()
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator
)

In [59]:
model_dtc_cv = dtc_cv.fit(train)

In [60]:
dtc_cv_preditcs_test = model_dtc_cv.transform(test)

In [61]:
print("Acurácia: %f" % evaluator.evaluate(dtc_cv_preditcs_test, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(dtc_cv_preditcs_test, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(dtc_cv_preditcs_test, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(dtc_cv_preditcs_test, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.786333
Precisão: 0.770122
Recall: 0.808108
F1: 0.788658
